# Mini Project: Dermatologist AI

## Introduction
In this notebook I will document my solution to Udacity's Mini Project: Dermatologist AI.  I will build a convolutional neural network to analyse images of lesions and then diagnose them. This project will take the image data from the [2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection](https://challenge.kitware.com/#challenge/583f126bcad3a51cc66c8d9a).  The result will be one of three different skin diseases: melanoma, nevus, or seborrheic keratosis.

The data was drawn from these three locations:

1. [Training](https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip)
2. [Validation](https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip)
3. [Testing](https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip)


## Plan
Sebastian Thrun and his team of students found that a pretrained network [performed better](https://classroom.udacity.com/nanodegrees/nd101/parts/b9c4c3c3-b524-427b-8832-9d0748f14a2e/modules/cb574ac4-7144-4ba5-97b9-1c1265525ff8/lessons/54e18898-2666-445d-ba5c-ecab62a61d00/concepts/3e74481c-c1f3-45c1-b0b5-e1cf479df45d) than an untrained network. I will use transfer learning from one of the following pre-trained networks: VGG-19, ResNet-50, Inception, or Xception. The best network results will be kept in this notebook.  I will report the results from the other networks.

I began by using a very simple 3 Node Dense layer and freezing the whole pre-trained network (except VGG19).  I initially tried keeping the last CONV block for VGG16, but it took too long to train, so I ended up freezing the whole thing.  This was to quickly find which one had the best accuracy on the test set.  Additionally, I used preloaded images and no Image Generator.

With only one 3 node DENSE(softmax) layer
- vgg19 test accuracy: 65.5000%
- vgg16 test accuracy: 65.5000%
- resnet test accuracy: 38.1667%
- xception test accuracy: 66.0000%
- inception test accuracy: 61.6667%

xception was the best pretrained network (by only 0.5%!), so I proceeded with that one and used an Image Generator.

With 2 Dense Layers: 512 (relu, no dropout), 3 (softmax)
- xception test accuracy: 65.5000%  (Discouraging and suspicious since that 65.5% accuracy keeps coming up!  I still don't know why.)

With 3 Dense layers : 1028 relu/dropout, 512 relu none, 3 (softmax)
- xception test accuracy: 67.0000% (aha! progress! I decided to run this through the official results.py)
> - Category 1 Score: 0.477
> - Category 2 Score: 0.742
> - Category 3 Score: 0.609

Not so good!  I'd be sending almost everyone to the lab!

The above were all done with a learning rate of 1e-4. I then did a learning rate of 1e-5
- xception test accuracy: 67.333% (aha! progress!)
> - Category 1 Score: 0.572
> - Category 2 Score: 0.793
> - Category 3 Score: 0.682

Better, but still needs improvement. For now, I will rest my case as I have RNN's to train...

## Load Data?
Depending on the 'fit' argument given below, the code will load the picture tensors into memory.

In [1]:
# import the necessary packages
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12.0, 10.0]

from keras.preprocessing import image
from tqdm import tqdm

from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    class_files = np.array(data['filenames'])
    class_targets = np_utils.to_categorical(np.array(data['target']), 3)
    return class_files, class_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')
last_inputShape = (0,0)

Using TensorFlow backend.


## Experimental code

*Note: This code seems to be all over the internet, I forget which page I orignally copied it from, but you can find it [here](https://github.com/hbhasin/Image-Recognition-with-Deep-Learning/blob/master/classify_butterfly_image.py). I'm also borrowing code from this great resource on* [Transfer Learning using Keras](https://towardsdatascience.com/transfer-learning-using-keras-d804b2e04ef8).  Later block(s) use another code snippet from the internet. Again, I've seen [this](https://towardsdatascience.com/transfer-learning-using-keras-d804b2e04ef8) in more than one place.


In [2]:
args = {'model': 'xception'
        ,'fit': 'fit_generator'
        ,'image':'donkey.jpg'}


# define a dictionary that maps model names to their classes
# inside Keras
MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception,  # TensorFlow ONLY
    "resnet": ResNet50
}
## MY CODE ##
FROZEN_LAYERS = {
    "vgg16": 22,  # Freezes everything
    "vgg19": 17,  # Freezes everything except the final Conv2D block
    "inception": 311,   #Don't understand it, so freezing whole thing. Final classification layers are not imported anyway.
    "xception": 132,  # TensorFlow ONLY, same note as for InceptionV3, don't understand it
    "resnet": 175  #ditto don't understand it
}

# esnure a valid model name was supplied via command line argument
if args["model"] not in MODELS.keys():
    raise AssertionError("The --model command line argument should "
                         "be a key in the `MODELS` dictionary")

# initialize the input image shape (224x224 pixels) along with
# the pre-processing function (this might need to be changed
# based on which model we use to classify our image)
inputShape = (224, 224)
input_shape = (224, 224, 3)
preprocess = imagenet_utils.preprocess_input

# if we are using the InceptionV3 or Xception networks, then we
# need to set the input shape to (299x299) [rather than (224x224)]
# and use a different image processing function
if args["model"] in ("inception", "xception"):
    inputShape = (299, 299)
    input_shape = (299,299,3)
    preprocess = preprocess_input

# load our the network weights from disk (NOTE: if this is the
# first time you are running this script for a given network, the
# weights will need to be downloaded first -- depending on which
# network you are using, the weights can be 90-575MB, so be
# patient; the weights will be cached and subsequent runs of this
# script will be *much* faster)
print("[INFO] loading {}...".format(args["model"]))
Network = MODELS[args["model"]]
model = Network(weights="imagenet", include_top=False, input_shape = input_shape)

## My Code
#from keras import applications
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
#from keras import backend as k 


## FREEZE LAYERS
for layer in model.layers[:FROZEN_LAYERS[args["model"]]]:
    layer.trainable = False

## Define my classification layers
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation="softmax")(x)
# 246,377,651 trainable params with 1200, 512, 3

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)


[INFO] loading xception...


In [3]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network
#print("[INFO] loading and pre-processing image...")
#image = load_img(args["image"], target_size=inputShape)
#image = img_to_array(image)

# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through thenetwork
#image = np.expand_dims(image, axis=0)

# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
#image = preprocess(image)


def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=inputShape)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3) or (299, 299, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    img = np.expand_dims(x, axis=0)
    return preprocess(img)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

# pre-process the data for Keras
if args['fit']=='fit':
    loaded = False
    try:
        q = train_tensors[0]
        loaded = True
    except:
        pass
    if (loaded == False) | (last_inputShape != inputShape):
        train_tensors = paths_to_tensor(train_files).astype('float32')
        valid_tensors = paths_to_tensor(valid_files).astype('float32')
        #test_tensors = paths_to_tensor(test_files).astype('float32')  #These are loaded below.
        last_inputShape = inputShape
    else:
        #test_tensors = paths_to_tensor(test_files).astype('float32')
        last_inputShape = inputShape

# classify the image
#print("[INFO] classifying image with '{}'...".format(args["model"]))
#preds = model.predict(image)
#P = imagenet_utils.decode_predictions(preds)

# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
#for (i, (imagenetID, label, prob)) in enumerate(P[0]):
#    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

# load the image via OpenCV, draw the top prediction on the image,
# and display the image to our screen
#orig = cv2.imread(args["image"])
#(imagenetID, label, prob) = P[0][0]
#cv2.putText(orig, "Label: {}, {:.2f}%".format(label, prob * 100),
#	(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
#cv2.imshow("Classification", orig)
#cv2.waitKey(0)


# convert BGR image to RGB for plotting
#cv_rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
#plt.imshow(cv_rgb)
#plt.title('Classy')
#plt.show()

In [4]:
for i, layer in enumerate(model_final.layers):
    print(i, layer.name, layer.trainable)
model_final.summary()

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14 batch_normalization_1 False
15 add_1 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_2 False
23 block3_pool False
24 batch_normalization_2 False
25 add_2 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_3 False
33 block4_pool False
34 batch_normalization_3 False
35 add_3 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2

## Hyper Parameters
Modify these to help the model learn (lr = learning rate).  Also, lowering the ```batch_size``` can help with memory constaints. Especially if you get those obscure TensorFlow errors.  For example ```OOM when allocating tensor with shape[2048,1024]```. That one actually makes some sense!

Increasing the the batch size can speed up the model some, but memory is the main constraint.  If the batch size gets too large (>500) the model may have not learn as well.

In [5]:
train_data_dir = "data/train"
validation_data_dir = "data/valid"
batch_size = 20
epochs = 500
lr = 1e-5

zoom_range = 0.3
shift_range = 0.2
rotation_range = 30

import os
def filecount(dir_name):
# return the number of files in directory dir_name
    try:
        return len([f for f in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, f))])
    except:
        return 0


dirs = os.listdir(train_data_dir)
nb_train_samples = 0
for dir_name in dirs:
    nb_train_samples += filecount(os.path.join(train_data_dir, dir_name))

dirs = os.listdir(validation_data_dir)
nb_validation_samples = 0
for dir_name in dirs:
    nb_validation_samples += filecount(os.path.join(validation_data_dir, dir_name))

print(nb_train_samples)
print(nb_validation_samples)

2000
150


### Compile the model

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
# compile the model
opt = optimizers.RMSprop(lr=lr)
model_final.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])


### Train (Fit) the Model
This is the code block that can take hours to run...

In [7]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./inputShape[0],
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = zoom_range,
width_shift_range  = shift_range,
height_shift_range = shift_range,
rotation_range     = rotation_range)

test_datagen = ImageDataGenerator(
rescale = 1./inputShape[0],
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = zoom_range,
width_shift_range  = shift_range,
height_shift_range = shift_range,
rotation_range     = rotation_range)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = inputShape,
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = inputShape,
class_mode = "categorical")

# Save the model according to the conditions
modelCheckpointName = 'saved_models/' + args["model"] + '_1.hdf5'
checkpoint = ModelCheckpoint(modelCheckpointName, monitor='val_loss', verbose=2, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='auto')


# Train the model
if args['fit']=='fit_generator':
    hist = model_final.fit_generator(train_generator,
        steps_per_epoch = nb_train_samples // batch_size,     epochs = epochs,
        validation_data = validation_generator, validation_steps = nb_validation_samples // batch_size,
        callbacks = [checkpoint, early])
else:
    hist = model_final.fit(train_tensors, train_targets, 
              validation_data=(valid_tensors, valid_targets),
              epochs=epochs, batch_size=batch_size, callbacks=[checkpoint, early], verbose=2)

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Epoch 1/500


ResourceExhaustedError: OOM when allocating tensor with shape[204800,1200]
	 [[Node: mul_8 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](lr/read, gradients/dense_1/MatMul_grad/MatMul_1)]]

Caused by op 'mul_8', defined at:
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "D:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\zmq\eventlo

### Plot the History
This can show us if the model is having problems overfitting or not learning.

In [ ]:
# ['acc', 'loss', 'val_acc', 'val_loss']
yl = hist.history['loss']
ya = hist.history['acc']
yvl = hist.history['val_loss']
yva = hist.history['val_acc']
x = np.arange(len(yl))

plt.close('all')
f, axarr = plt.subplots(2, sharex=True)

axarr[0].set_title('History')
axarr[0].plot(x,yl, label='Loss')
axarr[0].plot(x,yvl, label='Val Loss')
#axarr[0].xlabel('Epochs')
#axarr[1].ylabel('Loss')
axarr[1].plot(x,ya, label='Accuracy')
axarr[1].plot(x,yva, label='Val Accuracy')
#axarr[1].xlabel('Epochs')
#axarr[1].ylabel('Accuracy')
axarr[0].legend(loc=1)
axarr[1].legend(loc=1)
plt.show()

### Load/Reload Test Tensors and Gauge Test Accuracy
I have these code blocks here so I don't have to rerun the whole model to see results.

In [ ]:
import numpy as np
test_tensors = paths_to_tensor(test_files).astype('float32')
last_inputShape = inputShape

In [ ]:
modelCheckpointName = 'saved_models/' + args["model"] + '_1.hdf5'
#Load the model weights with the best validation loss.
print('Loading best weights back into model...')
model_final.load_weights(modelCheckpointName)

# Calculate classification accuracy on the test dataset.
# get index of predicted class for each image in test set
class_predictions = [np.argmax(model_final.predict(np.expand_dims(feature, axis=0)))\
                           for feature in test_tensors]
predictions = [model_final.predict(np.expand_dims(feature, axis=0)) for feature in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(class_predictions)==np.argmax(test_targets, axis=1))/len(class_predictions)
print(args["model"] + ' Test accuracy: %.4f%%' % test_accuracy)

### With only 1 3 node DENSE(softmax) layer
# vgg19 Test accuracy: 65.5000%
# vgg16 Test accuracy: 65.5000%
# resnet Test accuracy: 38.1667%
# xception Test accuracy: 66.0000%
# inception Test accuracy: 61.6667%

## With 2 Dense Layers: 512 (relu, no dropout), 3(softmax)
# xception Test accuracy: 65.5000%

## With 3 Dense layers : 1028 relu/dropout, 512 relu none, 3 softmax
# xception Test accuracy: 67.0000%

### Export Test Result for Udacity's Evaluation

In [ ]:
import pandas as pd
g = 0
print(test_files[g], predictions[g][0], class_predictions[g], test_targets[g])
melax = [t[0][0] for t in predictions]
servx = [t[0][2] for t in predictions]
d = {'Id': test_files, 'task_1':melax, 'task_2':servx}
df = pd.DataFrame(d)
df = df.sort_values(by=['Id'])
df.to_csv('sample_predictions2.csv')